<a href="https://colab.research.google.com/github/sulochandhungel/ReferenceET/blob/master/ReferenceET_for_USA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import ee
# Trigger the authentication flow.
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=PLORKKmiL0LhFbSBH0Lo43Cdl9_arEbF8dSF8iiFYRM&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 
Successfully saved authorization token.


In [0]:
# Import Earth Engine
import ee

try:
    ee.Initialize()
    print('The Earth Engine package initialized successfully!')
except ee.EEException as e:
    print('The Earth Engine package failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

The Earth Engine package initialized successfully!


In [0]:
import subprocess

try:
    import timezonefinder
except ImportError:
    print('timezonefinder package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'timezonefinder'])
print ("Done!")

Done!


In [0]:

# Function for ASCE Standardized Reference ET equation
def ASCE_hourly_ET(w_stn_lon, w_stn_lat,
                   year_, mo_, da_, hr_, min_, sec_,
                   T, RH, Rs, uz,
                   zw=2, stn_elev_m=None, t1=1.0, Lz=None, Lm=None,
                   short_ref_ETos = True, tall_ref_ETrs = False,
                   fcd_beta_gt_0_3 = None,
                   verbose=True):
    '''
    w_stn_lon = Longitude of Weather Stn
    w_stn_lat = Latitude of Weather Stn
    year_ = year of weather data
    mo_ = month of weather data
    da_ = day of weather data
    hr_ = hour of weather data
    min_ = minute of weather data
    sec_ = second of weather data
    T = Temperature in deg C
    RH = Relative Humidity in %
    Rs = Incoming solar radiation in MJ m^-2 hr^-1
    uz = windspeed in m/s
    zw = measurement height of wind speed
    stn_elev = Elevation of weather station
    t1 = calculation period
    Lz = longitude of the center of the local time zone [expressed as +ve degrees west of Greenwich].
         In the US,  Lz = 75, 90, 105 and 120 for Eastern, Central, Rocky Mountain, and Pacific Time zones
    Lm = longitude of solar radiation measurement site[expressed as +ve degrees west of Greenwich]
    fcd = cloudiness function [dimensionless] (limited to 0.05 <= fcd <= 1.0)
    '''
    # Required functions
    import numpy
    import numpy as np
    import pandas as pd
    import datetime
    
    pd_dt = pd.Timestamp(year_, mo_, da_, hr_, min_, sec_) #'%Y-%m-%d %H:%M:%S'
    dt_str = pd_dt.strftime('%Y-%m-%d %H:%M:%S')
    
    def calc_es(T):
        es = 0.6108 * np.exp((17.27 * T) / (T + 237.3))  # T is in deg C
                                                         # es is in kPa
        return (es)

    def calc_ea_3(T, RH):
        es = calc_es(T)
        ea = es * RH/100 # RH is in percent
                     # ea is in kPa
        return (ea)

    def calc_SVP_vs_T_slp(T):
        delta = (2503 * np.exp((17.27 * T)/(T + 237.3))) / np.power((T + 237.2),2) # T is in deg C
                                                                                   # delta is in kPa/deg C
        return (delta)

    def calc_P(z):
        P = 101.3 * np.power(((293 - (0.0065 * z))/293), 5.26) # z in m
                                                               # P in kPa
        return (P)

    def norm_DT(lon, lat, dt_str):
        ''' Normalizes datetime taking care of daylight savings for hour angle
        Please use date string format as '%Y-%m-%d %H:%M:%S'
        '''
        from pytz import timezone
        import pytz

        from timezonefinder import TimezoneFinder
        tf = TimezoneFinder()
    
        w_stn_tz = timezone(tf.timezone_at(lng=lon, lat=lat))
        #print (w_stn_tz)
        
        t = datetime.datetime.strptime(dt_str, '%Y-%m-%d %H:%M:%S')
        localized_DT = w_stn_tz.localize(t)
        #print (localized_DT)
        #norm_DT = w_stn_tz.normalize(localized_DT)
        return (localized_DT + datetime.timedelta(hours=1) -localized_DT.dst())
    
    def getElev(w_stn_lon, w_stn_lat):
        loc_geom = ee.Geometry.Point(w_stn_lon, w_stn_lat)
        z = ee.Image("USGS/NED").clip(loc_geom).reduceRegion(ee.Reducer.mean()).getInfo().get('elevation')#.values()[0]
        if z is None:
            z = ee.Image("USGS/SRTMGL1_003").clip(loc_geom).reduceRegion(ee.Reducer.mean()).getInfo().get('elevation')#.values()[0]
        return (z)
    
    def find_Lz(w_stn_lon, w_stn_lat,
                #tzs_id = 'ft:1zFYAWcrUQp0eMX-17kWniaJ2qZoZRDWWQlR4fVfh'):
                tzs_id = "users/sulochandhungel/TimeZone_SD"):
        ''' Lz = longitude of the center of the local time zone [expressed as +ve degrees west of Greenwich].
            In the US,  Lz = 75, 90, 105 and 120 for Eastern, Central, Rocky Mountain, and Pacific Time zones'''

        
        loc_geom = ee.Geometry.Point(w_stn_lon, w_stn_lat)
        tzs = ee.FeatureCollection(tzs_id)
        sel_tz = ee.Feature(tzs.filterBounds(loc_geom).first())
        sel_tz_cent = sel_tz.centroid()
        Lz = sel_tz_cent.getInfo().get('geometry').get('coordinates')[0]
        if Lz>=-180 and Lz<=0:
            Lz = Lz * -1
        elif Lz>=0 and Lz<=180:
            Lz = (Lz * -1)+180
            
        return (Lz)
    
    def find_Lm(w_stn_loc):
        '''Lm = longitude of solar radiation measurement site[expressed as +ve degrees west of Greenwich]'''
        if w_stn_lon>=-180 and w_stn_lon<=0:
            Lm = w_stn_lon * -1
        elif w_stn_lon>=0 and w_stn_lon<=180:
            Lm = (w_stn_lon * -1)+180
        return (Lm)

    
    def find_Sc(J):
        '''# Sc = seasonal correction for solar time'''
        import numpy as np
        # Sc = 0.1645 * sin(2b) - 0.1255 * cos(b) - 0.025 * sin(b)
        # b = 2 * pi * (J - 81)/364
        # b => radians
        b = 2.0 * np.pi * (J - 81.0)/364.0
        Sc = (0.1645 * np.sin(2*b)) - (0.1255 * np.cos(b)) - (0.025 * np.sin(b))
        return (Sc)
    
    def find_hour_angle(t, Lz, Lm, Sc, t1, delta_lc, phi):
        
        omega = (np.pi/12) * ((t + (0.06667 * (Lz - Lm)) + Sc) - 12)
        omega1 = omega - (np.pi * t1/24)
        #print ('real_omega1 = ' + str(omega1))
        
               
        # omega2 = solar time angle at the end of each period
        omega2 = omega + (np.pi * t1/24)
        #print ('real_omega2 = ' + str(omega2))
        
        # omega_s = sunset hour angle
        # omega_s = arccos[-tan(phi)*tan(delta_lc)]
        
        # phi = latitude (radians)
        phi = w_stn_lat * np.pi/180
        omega_s = np.arccos(-1.0 * np.tan(phi) * np.tan(delta_lc))
        #print ('real_omega_s = ' + str(omega_s))
        
        #print (omega_s)
    
        # For solar time angles which are after sunset and before sunrise,
        if omega1 < (-1 * omega_s):
            #print ('omega1 < (-1 * omega_s)')
            omega1 = (-1 * omega_s)
        elif omega1 > omega_s:
            #print ('omega1 > omega_s')
            omega1 = omega_s
        
        if omega2 < (-1 * omega_s):
            #print ('omega2 < (-1 * omega_s)')
            omega2 = (-1 * omega_s)
        elif omega2 > omega_s:
            #print ('omega2 > omega_s')
            omega2 = omega_s
    
        if omega1 > omega2:
            #print ('omega1 > omega2')
            omega1 = omega2
        return [{'omega': omega,
                 'omega_s': omega_s,
                 'omega1': omega1,
                 'omega2': omega2}]
    # FUNCTIONS END -------------------
    
    
    # CONSTANTS (Start) -------------------------------------------
    # Constants

    daily = False
    hourly_day = True
    hourly_night = False

    if short_ref_ETos:
        ET_type = "ETos"
        if hourly_day:
            Cn = 37 
            Cd = 0.24
            G_hr_multiplier = 0.1
        if hourly_night:
            Cn = 37
            Cd = 0.96
            G_hr_multiplier = 0.5
        if daily:
            Cn = 900
            Cd = 0.34

    if tall_ref_ETrs:
        ET_type = "ETrs"
        if hourly_day:
            Cn = 66 
            Cd = 0.25
            G_hr_multiplier = 0.04
        if hourly_night:
            Cn = 66
            Cd = 1.7
            G_hr_multiplier = 0.2
        if daily:
            Cn = 1600
            Cd = 0.38

    # albedo = surface albedo
    surfalbedo = 0.23
    
    # CONSTANTS (End) -------------------------------------------
    
    
    # User Input Data (Start)-----------------------------------

    # Weather data
    # T = Air Temperature
    # T units => deg C

    #T = 17.65 # ---- Weather INPUT 1 ----

    # RH = Relative Humidity
    # RH units => percent

    #RH = 56 # ---- Weather INPUT 2 ----

    # Rs = Measured incoming solar Radiation
    # Rs units => MJ m^-2 hr^-1
    # Other units are => SlrkW_Avg = Kilowatts per sq. meter
    #                    W m^-2 = SlrkW_Avg * 1000
    
    #                 => SlrMJ_Tot = Daily Total Flux
    #                    W m^-2 = SlrMJ_Tot * 11.574

    #                 => MJ m^-2 hr^-1 = W m^-2 * 0.0036
    
    #Rs = 0.8905 * 1000 * 0.0036 # ---- Weather INPUT 3 ----

    #uz = 2 # m/s
    #zw = 2 # m
    
    #z = 0 #m

    # t1 = length of calculation period
    # unit of t1 => hours
    #t1 = 1.0


    # User Input Data (End)-----------------------------------
    
    # 1 # P = Atmospheric Pressure
    # P units => kPa
    
    if stn_elev_m is None:
        z = getElev(w_stn_lon, w_stn_lat)
    else:
        z = stn_elev_m
    
    P = calc_P(z)

    # 2 # gamma = Psychometric Constant 
    # gamma units => kPa/deg C
    gamma = 0.000665 * P
    
    # 3 # delta = Slope of SVP vs. Temp curve
    # delta units => kPa / deg C
    delta = calc_SVP_vs_T_slp(T)
    
    # 4 # es = Saturated Vapor Pressure
    es = calc_es(T)
    
    # 5 # ea = vapor Pressure
    # ea units => kPa
    
    # Method preference:
    # 1 - ea averaged over period 
    # 2 - measured or calculated dew point temperature averaged over period
    # 3 - Average RH and T for the hour
    # 4 - Wet-bulb and dry-bulb temperature
    # 5 - Daily minimum air temperature
    use_ea_method = 3
    if use_ea_method ==3:
        ea = calc_ea_3(T, RH)

    # Rn = Rns - RnL
    
    # 6 # Rns = net solar or short-wave radiation
    # Rns units => MJ m^-2 hr^-1
    Rns = (1 - surfalbedo) * Rs
    
    # RnL = sigma * fcd * (0.34 - 0.14 * sqrt(ea)) * (T_K_hr ^ 4)
    # sigma = Stefan-Boltzmann constant
    # sigma units => MJ K^-4 m^-2 hr^-1
    sigma = 2.042E-10

    # fcd = cloudiness function [dimensionless] (limited to 0.05 <= fcd <= 1.0)
    # fcd = (1.35 * (Rs/Rso)) - 0.35
    
    # rel_sr = Rs/Rso = relative solar radiation (limited to 0.3 <= Rs/Rso <= 1.0)
    # Rs = measured/calculated solar radiation 
    # Rso = calculated clear-sky radiation 
    
    # Rso = (0.75 + (2E-5) * z) * Ra
    # z = elevation in m
    # Rso units => MJ m^-2 hr^-1
    
    # Ra = extraterrestrial radiation 
    # Ra units => MJ m^-2 hr^-1
    
    # Ra = (12/pi) * Gsc * dr * [(omega2 - omega1) * sin(phi) * sin(delta_lc) +\
    #                           cos(phi) * cos(delta_lc) * (sin(omega2) - sin(omega1))]
    # Gsc = solar constant
    # Gsc units => MJ m^-2 hr^-1
    Gsc = 4.92

    # dr = inverse relative distance factor(squared) for the earth-sun
    # dr units => unitless
    # dr = 1 + 0.033 * cos((2 * pi/365) * J)
    # J = Julian Day
    J = pd_dt.dayofyear
    dr = 1 + 0.033 * np.cos((2 * np.pi/365) * J)

    # delta_lc = lowercase delta
    # delta_lc = solar declination [radians]
    # delta_lc = 0.409 * sin((2*pi/365)*J - 1.39)
    delta_lc = 0.409 * np.sin(((2 * np.pi/365) * J) - 1.39)
    
    # omega1 = solar time angle at the beginnning of each period
    # omega1 = omega + (pi * t1/24)
    
    # omega = solar time angle at the midpoint of the period
    # omega = (pi/12) * [(t + 0.06667 * (Lz - Lm) + Sc) - 12]
    
    t_DT = norm_DT(w_stn_lon, w_stn_lat, dt_str)
    #print (dt_str)
    #t_DT = datetime.datetime.strptime(dt_str, '%Y-%m-%d %H:%M:%S')
    t = t_DT.hour + (t_DT.minute/60.0) + (t_DT.second/3600.0)
    #print (t)
    
    # t = decimal Time
    # Normalized time 
    
    # Lz = longitude of the center of the local time zone [expressed as +ve degrees west of Greenwich].
    #      In the US,  Lz = 75, 90, 105 and 120 for Eastern, Central, Rocky Mountain, and Pacific Time zones
    if Lz is None:
        Lz = find_Lz(w_stn_lon, w_stn_lat)

    # Lm = longitude of solar radiation measurement site[expressed as +ve degrees west of Greenwich]
    if Lm is None:
        Lm = find_Lm(w_stn_lon)
    
    # Sc = seasonal correction for solar time
    Sc = find_Sc(J)
    phi = w_stn_lat * np.pi/180
    
    hr_angles = find_hour_angle(t, Lz, Lm, Sc, t1, delta_lc, phi)
    omega = hr_angles[0].get('omega')
    omega_s = hr_angles[0].get('omega_s')
    omega1 = hr_angles[0].get('omega1')
    omega2 = hr_angles[0].get('omega2')
    
    
    #print ('t = ' + str(t))
    #print ('Lz = ' + str(Lz))
    #print ('Lm = ' + str(Lm))
    #print ('J = '+ str(J))
    #print ('Sc = ' + str(Sc))
    
    
    #print ('omega1 = ' + str(omega1))
    #print ('omega2 = ' + str(omega2))
    #print ('omega = ' + str(omega))
    #print ('omega_s = ' + str(hr_angles[0].get('omega_s')))
    
    
    #print ('phi = ' + str(phi))
    #print ('delta_lc = ' + str(delta_lc))
    #print (Gsc)
    #print (dr)
    omega_del = (omega2 - omega1)
    #(12.0/np.pi)
    Ra = (12.0/np.pi) * Gsc * dr * ((omega_del * np.sin(phi) * np.sin(delta_lc)) +\
                                (np.cos(phi) * np.cos(delta_lc) * (np.sin(omega2) -\
                                np.sin(omega1))))
    #print ('dr = ' + str(dr))
    #print ('lat = ' + str(w_stn_lat))
    #print ('lat = ' + str(phi))
    #print ("Ra = " + str(Ra))
    #print ("Ra = " + str(Ra/0.0036))
    
    Rso = (0.75 + (2E-5) * z) * Ra
    
    rel_sr = Rs/Rso

    if (rel_sr)<=0.3:
        rel_sr = 0.3
    elif (rel_sr)>1.0:
        rel_sr = 1.0
    
    fcd = (1.35 * rel_sr) - 0.35
    
    beta = np.arcsin ((np.sin(phi) * np.sin(delta_lc)) + (np.cos(phi) * np.cos(delta_lc) * np.cos(omega)))
    #if beta < 0.3:
    #    if fcd_beta_gt_0_3 is None:
    #        return "Need fcd for beta gt 0.3"
    #    else:
    #        fcd = fcd_beta_gt_0_3
    
    RnL = sigma * fcd * (0.34 - 0.14 * np.sqrt(ea)) * (np.power((T+273.3), 4))
    
    Rn = Rns - RnL
    
    # G = Soil Heat Flux
    if short_ref_ETos:
        if Rn<0:
            G = 0.5 * Rn
        else:
            G = 0.1 * Rn
    
    if tall_ref_ETrs:
        if Rn<0:
            G = 0.2 * Rn
        else:
            G = 0.04 * Rn
            
    u2 = uz * 4.87 / np.log((67.8 * zw) - 5.42)

    ETsz = ((0.408 * delta * (Rn - G)) +\
            (gamma * (Cn/(T+273)) * u2 * (es-ea)))/(delta + (gamma * (1+(Cd * u2))))
    
    if verbose:
        print ("Date and Time = " + dt_str)
        print ("Day of the Year = " + str(J))
        print ("Decimal Time (t) = %.5f" %t)
        
        print ("Longitude of the center of the local time zone (Lz) = %.3f degrees" %Lz)
        print ("Longitude of solar radiation measurement site (Lm) = %.3f degrees" %Lm)
        print ("Seasonal correction for solar time (Sc) = %.3f hours" %Sc)
        print ("Solar time angle at the midpoint of the period (omega) = %.3f rads" %omega)
        print ("Sunset hour angle (omega_s) = %.3f rads" %omega_s)
        print ("Solar time angle at the beginnning of each period (omega1) = %.3f rads" %omega1)
        print ("Solar time angle at the end of each period (omega2) = %.3f rads" %omega2)
        print ("Solar declination (delta_lc) = %.3f radians" %delta_lc)
        print ("Inverse relative distance factor(squared) for the earth-sun (dr) = %.3f" %dr)
        print ("Angle of the sun above the horizon (beta) = %.3f" %beta)
        
        print ("Air Temperature (T) = %.2f deg C" %T)
        print ("Relative Humidity (RH) = %.0f %%" %RH)
        print ("Elevation (z) = %.1f m" %z)
        print ("Wind Speed measured at (zw) = %.1f m" %zw)
        print ("Wind Speed at {} m (uz) = {} m/s".format(*[zw, uz]))
        print ("Wind Speed at 2.0 m (u2) = %.1f m/s" %u2)
        print ("Atmospheric Pressure (P) = %.2f kPa" %P)
        print ("Psychrometric Constant (gamma) = %.4f kPa/deg C" %gamma)
        print ("Saturated Vapor Pressure (es) = %.3f kPa" %es)
        print ("Vapor Pressure (ea) = %.3f kPa" %ea)
        print ("Slope of SVP Vs. Temp curve (delta) = %.3f kPa/deg C" %delta)

        print ("Measured solar radiation (Rs) = %.3f MJ m^-2 hr^-1" %Rs)
        print ("Calculated clear-sky radiation (Rso) = %.3f MJ m^-2 hr^-1" %Rso)
        print ("Relative solar radiation (rel_sr = Rs/Rso) = %.3f" %rel_sr)  
        print ("Cloudiness function (fcd) = %.3f" %fcd)

        print ("Extraterrestrial radiation (Ra) = %.3f MJ m^-2 hr^-1" %Ra)
        print ("Net solar or short-wave radiation (Rns) = %.3f MJ m^-2 hr^-1" %Rns)
        print ("Net outgoing long-wave radiation (RnL) = %.3f MJ m^-2 hr^-1" %RnL)
        print ("Net radiation (Rn) = %.3f MJ m^-2 hr^-1" %Rn)
        print ("Soil Heat Flux (G) = %.3f MJ m^-2 hr^-1" %G)
        print ("Standardized reference crop evapotranspiration (" + ET_type + ") = %.3f mm/hr" %ETsz)
    
    
    return {'ETsz': ETsz,
            'G': G,
            'Rn': Rn,
            'Rns': Rns,
            'RnL': RnL,
            'rel_sr': rel_sr,
            'fcd': fcd,
            'Rs': Rs,
            'Rso': Rso,
            'Ra': Ra,
            'omega': omega,
            'omega1': omega1,
            'omega2': omega2,
            'omega_s': omega_s,
            'Sc': Sc,
            'Lm': Lm,
            'Lz': Lz,
            'normalized_DT': t_DT,
            'delta_lc': delta_lc,
            'Lz': Lz,
            'Lm': Lm
           }
ASCE_hourly_ET(w_stn_lon = -120.2771,
               w_stn_lat = 46.3464,
               year_ = 2016,
               mo_ = 5,
               da_ = 10,
               hr_ = 11,
               min_= 22,
               sec_ = 30,
               T = 19.69,
               RH = 31.5,
               Rs = 0.881,
               uz = 2.345)

Date and Time = 2016-05-10 11:22:30
Day of the Year = 131
Decimal Time (t) = 11.37500
Longitude of the center of the local time zone (Lz) = 119.021 degrees
Longitude of solar radiation measurement site (Lm) = 120.277 degrees
Seasonal correction for solar time (Sc) = 0.062 hours
Solar time angle at the midpoint of the period (omega) = -0.169 rads
Sunset hour angle (omega_s) = 1.915 rads
Solar time angle at the beginnning of each period (omega1) = -0.300 rads
Solar time angle at the end of each period (omega2) = -0.038 rads
Solar declination (delta_lc) = 0.311 radians
Inverse relative distance factor(squared) for the earth-sun (dr) = 0.979
Angle of the sun above the horizon (beta) = 1.054
Air Temperature (T) = 19.69 deg C
Relative Humidity (RH) = 32 %
Elevation (z) = 223.1 m
Wind Speed measured at (zw) = 2.0 m
Wind Speed at 2 m (uz) = 2.345 m/s
Wind Speed at 2.0 m (u2) = 2.3 m/s
Atmospheric Pressure (P) = 98.69 kPa
Psychrometric Constant (gamma) = 0.0656 kPa/deg C
Saturated Vapor Pressur

{'ETsz': 0.2657012060753057,
 'G': 0.06600799425821889,
 'Lm': 120.2771,
 'Lz': 119.02073375870143,
 'Ra': 4.179005478960321,
 'Rn': 0.6600799425821889,
 'RnL': 0.01829005741781106,
 'Rns': 0.67837,
 'Rs': 0.881,
 'Rso': 3.152900056776087,
 'Sc': 0.06193471701603692,
 'delta_lc': 0.3113041526109263,
 'fcd': 0.05500000000000005,
 'normalized_DT': datetime.datetime(2016, 5, 10, 11, 22, 30, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
 'omega': -0.1693389702840981,
 'omega1': -0.30023866418367284,
 'omega2': -0.03843927638452341,
 'omega_s': 1.9147962370965719,
 'rel_sr': 0.3}